In [1]:
#Solutions to questions 1 - 9
'''
/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */
	SELECT name
	FROM Facilities
	WHERE membercost <> 0.0
	LIMIT 0 , 30
	
	Tennis Court 1, Tennis Court 2, Massage Room 1, Massage Room 2, Squash Court

/* Q2: How many facilities do not charge a fee to members? */
	SELECT name
	FROM Facilities
	WHERE membercost = 0.0
	LIMIT 0 , 30
	
	4; Badminton Court, Table Tennis, Snooker Table, Pool Table

/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */
	SELECT facid, name, membercost, monthlymaintenance
	FROM Facilities
	WHERE membercost < monthlymaintenance * 0.2

/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */
	SELECT *
	FROM Facilities
	WHERE facid IN ( 1, 5 )

/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */
	SELECT name,
	CASE WHEN monthlymaintenance >100 THEN 'expensive'
		ELSE 'cheap' END AS monthlymaintenance
	FROM Facilities

/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */
	SELECT firstname, surname
	FROM Members
	WHERE memid = (
		SELECT MAX( memid )
		FROM Members )

/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */
	SELECT DISTINCT m.membername, f.name AS facility
	FROM Facilities AS f
	LEFT JOIN (
		SELECT CONCAT( m.firstname, ' ', m.surname ) AS membername, b.facid
		FROM Members AS m
		LEFT JOIN Bookings AS b
		USING ( memid )
		WHERE memid <>0
	) AS m
	USING ( facid )
	WHERE facid IN ( 0, 1 )
	ORDER BY m.membername

/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */
	SELECT f.name as facility, CONCAT( m.firstname, ' ', m.surname ) AS membername,
		CASE WHEN memid <> 0 THEN f.membercost * b.slots
		ELSE f.guestcost * b.slots END AS cost
	FROM Bookings AS b
	LEFT JOIN Members AS m
		USING (memid)
	LEFT JOIN Facilities as f
		USING (facid)
	WHERE b.starttime LIKE '2012-09-14%' AND (
		(memid <> 0 AND f.membercost * b.slots > 30.0)
		OR (memid = 0 AND f.guestcost * b.slots >30)
		)
	ORDER BY cost DESC

/* Q9: This time, produce the same result as in Q8, but using a subquery. */
	SELECT facility, membername, cost
	FROM (
		SELECT * , f.name AS facility, CONCAT( m.firstname, ' ', m.surname ) AS membername,
			CASE WHEN memid <>0	THEN f.membercost * b.slots
			ELSE f.guestcost * b.slots END AS cost
		FROM Bookings AS b
		LEFT JOIN Members AS m
			USING ( memid )
		LEFT JOIN Facilities AS f
			USING ( facid )
	) AS sub
	WHERE starttime LIKE '2012-09-14%' AND cost >30
	ORDER BY cost DESC
'''

"\n/* Q1: Some of the facilities charge a fee to members, but some do not.\nWrite a SQL query to produce a list of the names of the facilities that do. */\n\tSELECT name\n\tFROM Facilities\n\tWHERE membercost <> 0.0\n\tLIMIT 0 , 30\n\t\n\tTennis Court 1, Tennis Court 2, Massage Room 1, Massage Room 2, Squash Court\n\n/* Q2: How many facilities do not charge a fee to members? */\n\tSELECT name\n\tFROM Facilities\n\tWHERE membercost = 0.0\n\tLIMIT 0 , 30\n\t\n\t4; Badminton Court, Table Tennis, Snooker Table, Pool Table\n\n/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,\nwhere the fee is less than 20% of the facility's monthly maintenance cost.\nReturn the facid, facility name, member cost, and monthly maintenance of the\nfacilities in question. */\n\tSELECT facid, name, membercost, monthlymaintenance\n\tFROM Facilities\n\tWHERE membercost < monthlymaintenance * 0.2\n\n/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.\nT

In [2]:
import sqlite3
from sqlite3 import Error
import pandas as pd
 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print('sqlite3 version: ', sqlite3.version)
    except Error as e:
        print(e)
 
    return conn
 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query10 = """
        SELECT f.facid,
                f.name AS facility,
                (f.guestcost * g.guest_slots + f.membercost * m.member_slots) - (initialoutlay + 3*monthlymaintenance) AS total_revenue
        FROM Facilities AS f
        LEFT JOIN (
            SELECT facid, SUM(slots) AS member_slots
            FROM Bookings
            WHERE memid <> 0
            GROUP BY facid
            ) AS m
        USING (facid)
        LEFT JOIN (
            SELECT facid, SUM(slots) AS guest_slots
            FROM Bookings
            WHERE memid = 0
            GROUP BY facid
            ) AS g
        USING (facid)
        WHERE (f.guestcost * g.guest_slots + f.membercost * m.member_slots) - (initialoutlay + 3*monthlymaintenance) < 1000
        ORDER BY total_revenue DESC
        """
    cur.execute(query10)
 
    rows10 = cur.fetchall()
 
    df10 = pd.DataFrame(rows10)
    print(df10)
    print('\n')
    #for row in rows10:
    #    print(row)

#----------------
    query11 = """
        SELECT m.surname, m.firstname, s.surname AS recommenders_surname, s.firstname AS recommenders_firstname
        FROM Members AS m
        LEFT JOIN (
            SELECT memid, surname, firstname
            FROM Members
            WHERE memid <>0
        ) AS s ON m.recommendedby = s.memid
        WHERE recommendedby <> ' '
        ORDER BY m.surname, m.firstname
    """
    cur.execute(query11)
 
    rows11 = cur.fetchall()
 
    df11 = pd.DataFrame(rows11)
    print(df11)
    print('\n')
    #for row in rows11:
    #    print(row)
        
#-------------------
    query12 = """
        SELECT memid, surname, firstname, facid, facility, COUNT( facility ) as usage_rate
        FROM (
            SELECT b.bookid, b.facid, b.memid, f.name AS facility, m.surname, m.firstname
            FROM Bookings AS b
            LEFT JOIN Members AS m
                USING ( memid )
            LEFT JOIN Facilities AS f
                USING ( facid )
            WHERE memid <>0
        ) AS s
        GROUP BY memid, facility
        ORDER BY memid, facid
    """
    cur.execute(query12)
 
    rows12 = cur.fetchall()
 
    df12 = pd.DataFrame(rows12)
    print(df12)
    print('\n')
    #for row in rows12:
    #    print(row)

#------------
    query13 = """
        SELECT month, facid, facility, COUNT(facility) AS usage_rate
        FROM (
            SELECT f.name as facility, b.facid, strftime('%m', b.starttime) AS month
            FROM Bookings as b
            LEFT JOIN Facilities as f
                USING (facid)
        ) AS s
        GROUP BY month, facility
        ORDER BY month, facid
    """
    cur.execute(query13)
 
    rows13 = cur.fetchall()
    
    df13 = pd.DataFrame(rows13)
    print(df13)
    print('\n')
    #for row in rows13:
    #    print(row)

def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

sqlite3 version:  2.6.0
Query all tasks
   0                1       2
0  3     Table Tennis  -170.0
1  8       Pool Table  -175.0
2  7    Snooker Table  -255.0
3  2  Badminton Court -2243.5


                    0          1         2          3
0               Bader   Florence  Stibbons     Ponder
1               Baker       Anne  Stibbons     Ponder
2               Baker    Timothy   Farrell     Jemima
3              Boothe        Tim    Rownam        Tim
4             Butters     Gerald     Smith     Darren
5              Coplin       Joan     Baker    Timothy
6             Crumpet      Erica     Smith      Tracy
7                Dare      Nancy  Joplette     Janice
8             Farrell      David      None       None
9             Farrell     Jemima      None       None
10              GUEST      GUEST      None       None
11            Genting    Matthew   Butters     Gerald
12               Hunt       John   Purview  Millicent
13              Jones      David  Joplette     Janic